# Visualisation of masks for DICOM files in the dataset.

Observations
- These were different from those of NIFTI files converted using dicom2nifti which were rotated and indexed in a reverse order wrt slices
- These correspond much better to the annotation boxes
- There are a few boxes that are incorrectly labelled. Sequences with double-digit number of slices have an additional '0' in the simple paths dataset

In [43]:
%pip install pyradiomics dicom_numpy pydicom plotly matplotlib scikit-image simpleITK pynrrd dicom2nifti NiBabel NiLearn openpyxl pydicom-seg itkwidgets pandarallel

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Button, Slider
from matplotlib import patches, text, patheffects

import dicom_numpy
import pydicom as dicom

import dicom2nifti
import nibabel as nib
import nilearn as nil
import scipy.ndimage as ndi
import SimpleITK as sitk
import os

from radiomics import featureextractor

import re
from tqdm import tqdm
import itertools
from functools import wraps
import ipywidgets

from pandarallel import pandarallel

tqdm.pandas()
pandarallel.initialize(progress_bar=True,nb_workers= 20)

c:\ProgramData\Anaconda3\envs\mri_project\lib\site-packages\nilearn\__init__.py:67: FutureWarning: Python 3.7 support is deprecated and will be removed in release 0.12 of Nilearn. Consider switching to Python 3.9 or 3.10.
  _python_deprecation_warnings()


INFO: Pandarallel will run on 20 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [3]:
DATASET_PATH = '../../../Processed NIFTI Dataset'

SAVE_PATH = '../../../Segmentation visualisation'

In [4]:
annotation_boxes = pd.read_csv('../segmentation_annotations_NIFTI.csv')
annotation_boxes

Patient_ID  Start Row  End Row  Start Column  End Column  \
0    Breast_MRI_001        234      271           308         341   
1    Breast_MRI_002        251      294           108         136   
2    Breast_MRI_003        351      412            82         139   
3    Breast_MRI_004        262      280           193         204   
4    Breast_MRI_005        188      213           138         178   
..              ...        ...      ...           ...         ...   
917  Breast_MRI_918        345      395           338         395   
918  Breast_MRI_919        285      312           369         397   
919  Breast_MRI_920        172      193           337         355   
920  Breast_MRI_921        328      374           404         446   
921  Breast_MRI_922        258      270           149         164   

     Start Slice  End Slice  
0             89        112  
1             59         72  
2             96        108  
3             86         95  
4             76        122  
..           ...        ...  
917           62         85  
918           98        109  
919           87        101  
920           97        121  
921           82         92  

[922 rows x 7 columns]

## Extracting and saving bounding boxes for all patients


In [5]:
patients = os.listdir(DATASET_PATH)

In [6]:
try:
    os.removedirs(SAVE_PATH)

except:
    print("could not remove dir")

finally:
    print("created directory")
    os.makedirs(SAVE_PATH, 777, exist_ok=True)    

could not remove dir
created directory


In [11]:
def save_img(x):
    DATASET_PATH = '../../../Processed NIFTI Dataset'

    SAVE_PATH = '../../../Segmentation visualisation'

    import SimpleITK as sitk
    import os
    import matplotlib.pyplot as plt
    from matplotlib import patches, text, patheffects


    patient_id = x['Patient_ID']

    patient_dir = os.path.join(DATASET_PATH, patient_id)
    row1 = x['Start Row']
    row2 = x['End Row']

    col1 = x['Start Column']
    col2 = x['End Column']

    slice1 = x['Start Slice']
    slice2 = x['End Slice']

    # middle slice
    mid_slice = (slice1 + slice2)//2                                                                                # num slices - current slice

    slices_computing = {'starting_slice' : slice1, 
                        "middle_slice": mid_slice,
                        'ending_slice': slice2}

    fig, ax = plt.subplots(2, 3, figsize = (27, 18))                                                                # plotting

    r = 0

    
    segmentation_path = os.path.join(patient_dir, 'segmentation.img.gz')                                            # read nifti seq for segment
    postcontrast_path = os.path.join(patient_dir, 'post_1.img.gz')                                                  # postcontrast-1

    # try:
    segmentation_data = sitk.GetArrayFromImage(sitk.ReadImage(segmentation_path))                                   # get pixel data for segmentation
    postcontrast_data = sitk.GetArrayFromImage(sitk.ReadImage(postcontrast_path))                                   # get pixel data for postcontrast

    for slice_title, current_slice in slices_computing.items():
        slice_name = " ".join(slice_title.split('_')).title()                                       # printing info
        # precontrast
        
        ax[0][r].imshow(segmentation_data[current_slice, :, :], cmap = 'gray')
        ax[0][r].set_title(f"Segmentation [{slice_name}, Slice : {current_slice}]")
        ax[0][r].add_patch(patches.Rectangle((col1, row1), col2-col1, row2-row1, fill = False, edgecolor = 'red', lw = 2))

        ax[1][r].imshow(postcontrast_data[current_slice, :, :], cmap = 'gray')
        ax[1][r].set_title(f"Post-1 [{slice_name}, Slice: {current_slice}]")
        ax[1][r].add_patch(patches.Rectangle((col1, row1), col2-col1, row2-row1, fill = False, edgecolor = 'red', lw = 2))

        r+=1

    fig.suptitle("{}\nThe red box denotes the bounding box for the tumour as per the dataset".format(patient_id))
        # saving
    final_path = os.path.join(SAVE_PATH, "{}.png".format(patient_id))
    fig.savefig(final_path, facecolor = 'white')



annotation_boxes.parallel_apply(save_img, axis = 1)

0      None
1      None
2      None
3      None
4      None
       ... 
917    None
918    None
919    None
920    None
921    None
Length: 922, dtype: object